##### Note on running the notebook

What is currently a bit strange, is that this code only works once. If you want to let this notebook run for the second time, you seem to have to restart the notebook (Kernel->restart). This is not the case if you put all the code into a python file. 


### Looking in the detail of how TensorFlow opimizes the problem
The computation of 
$$
    y = a x + b 
$$

is encoded in the follwong TensorFlow graph, which reads in x_data and y_data and does some processing. The nodes of the graph, are operations called *ops*. For the documemation: "An op
takes zero or more `Tensors`, performs some computation, and produces zero or
more `Tensors`.  A `Tensor` is a typed multi-dimensional array." In our example, the Tensor `x_data` is a simple vector of size N.


In this notebook we investigate, who to write such a graph and also include additional information, while the graph is processed.
![graph.png, width=140](graph.png)

In [1]:
# Creating the data
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
N = 30
x_data = (np.linspace(0,10,N)).astype('float32')
y_data = (2.42 * x_data + 0.42 + np.random.normal(0,1,N)).astype('float32')

In [2]:
a = tf.Variable(1.0, name = 'a') #Note that 1.0 is needed
b = tf.Variable(0.01, name = 'b')
x = tf.placeholder('float32', [N], name='x_data')
y = tf.placeholder('float32', [N], name='y_data')


resi = a*x + b - y
loss = tf.reduce_sum(tf.square(resi))
init_op = tf.initialize_all_variables() #Initialization

train_op = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)
epochs = 1000

tf.name_scope("fitting")
loss_summary = tf.scalar_summary("loss_summary", loss) #<---
resi_summart = tf.histogram_summary("resi_summart", resi)
merged_summary_op = tf.merge_all_summaries()   #<-----  before the session
with tf.Session() as sess:
    sess.run(init_op)
    writer = tf.train.SummaryWriter("/tmp/dumm", sess.graph_def, 'graph.pbtxt')
    for e in range(epochs): #Fitting the data for 10 epochs
        sess.run(train_op, feed_dict={x:x_data, y:y_data})
        if (e < 5 | e > epochs - 5):
            print("epoch {} {}".format(e, sess.run(loss, feed_dict={x:x_data, y:y_data})))
        sum_str = sess.run(merged_summary_op, feed_dict={x:x_data, y:y_data})
        writer.add_summary(sum_str, e)
    res = sess.run([loss, a, b], feed_dict={x:x_data, y:y_data})
    print(res)
print('Finished all')

epoch 992 26.6808223724
epoch 993 26.6804332733
epoch 994 26.6800498962
epoch 995 26.6796646118
epoch 996 26.6792755127
epoch 998 26.6785106659
[26.678135, 2.3533804, 0.68155742]
Finished all


The graph is created and can be visualized using:
```
    tensorboard --logDir /tmp/dumm
```
and explored using the browser with the URL: http://0.0.0.0:6006